## Homework 1 -  Frequent Pattern Analysis
***
**Name**: $<$insert name here$>$ 
***

Remember that you are encouraged to discuss the problems with your instructors and classmates, but **you must write all code and solutions on your own**.

The rules to be followed for the assignment are:

- Do **NOT** load additional packages beyond what we've shared in the cells below.
- Some problems with code may be autograded.  If we provide a function or class API **do not** change it.
- Do not change the location of the data or data directory.  Use only relative paths to access the data.

In [1]:
import argparse
import pandas as pd
import numpy as np
import random 
import pickle5 as pickle
from pathlib import Path
from collections import defaultdict

### [10 points] Problem 1 - Apriori Implementation
***

A sample dataset has been provided to you in the './data/dataset.pickle' path. Here are the attributes for the dataset. Use this dataset to test your functions.

- Dataset should load the transactions in the form of a python dictionary where each key holds the transaction id and the value is a python list of the items purchased in that transaction. 
- An example transaction will have the following structure. If items A, C, D, F are purchased in transaction T3, this would appear as follows in the dictionary.

```python
transactions = {
   "T3": ["A", "C", "D", "F"]
}
```

Note:
- A sample dataset to test your code has been provided in the location "./data/dataset.pickle". Please maintain this as it would be necessary while grading.
- Do not change the variable names of the returned values.
- After calculating each of those values, assign them to the corresponding value that is being returned.

- If you are encountering any errors while loading the dataset, the following lines of code should help. Please delete the cells before submitting, to reduce any potential autograder issues.

```python
!pip install pickle5

import pickle5 as pickle
```

In [33]:
import itertools
  
def findsubsets(s, n):
    
#   A helper function that you can use to list of all subsets of size n. Do not make any changes to this code block.
#   Input: 
#       1. s - A python list of items
#       2. n - Size of each subset
#   Output:
#       1. subsets - A python list containing the subsets of size n.
    
    subsets = list(sorted((itertools.combinations(s,n))))
    return subsets

In [34]:
def items_from_frequent_itemsets(frequent_itemset):

#   A helper function that you can use to get the sorted items from the frequent itemsets. Do not make any changes
#   to this code block
#   Input:
#       1. Frequent Itemsets
#   Output:
#       1. Sorted list of items

    items = list()
    for keys in frequent_itemset.keys():
        for item in list(keys):
            items.append(item)
    return sorted(list(set(items)))

In [61]:
def generate_frequent_itemsets(dataset, support, items, n=1, frequent_items={}):
    
#   Input:
#       1. dataset - A python dictionary containing the transactions.
#       2. support - A floating point variable representing the min_support value for the set of transactions.
#       3. items - A python list representing all the items that are part of all the transactions.
#       4. n - An integer variable representing what frequent item pairs to generate.
#       5. frequent_items - A dictionary representing k-1 frequent sets. 
#   Output:
#       1. frequent_itemsets - A dictionary representing the frequent itemsets and their corresponding support counts.
    
    len_transactions = len(dataset)
    
    min_support_count = int(support * len_transactions) 
    if n == 1:
        
        # your code here
        frequent_itemsets = {}
        for item in items:
            item_count = 0
            for transaction in dataset.values():
                if item in transaction:
                    item_count += 1
            if item_count/len_transactions >= support:
                frequent_itemsets[item] = item_count
        frequent_items = frequent_itemsets
        
    else:
        candidates = findsubsets(items, n)
        frequent_itemsets = {}
        for candidate in candidates:
            candidate_count = 0
            for transaction in dataset.values():
                if set(candidate).issubset(transaction):
                    candidate_count += 1
            if candidate_count/len_transactions >= support:
                frequent_itemsets[tuple(candidate)] = candidate_count
        frequent_items = frequent_itemsets
    
    return frequent_itemsets

In [62]:

# This cell has hidden test cases that will run after you submit your assignment. 


In [63]:
import unittest

class TestX(unittest.TestCase):
    def setUp(self):
        self.min_support = 0.5
        self.items = ['A', 'B', 'C', 'D', 'E']
        self.dataset = dict()
        self.dataset["T1"] = ['A', 'B', 'D']
        self.dataset["T2"] = ['A', 'B', 'E']
        self.dataset["T3"] = ['B', 'C', 'D']
        self.dataset["T4"] = ['B', 'D', 'E']        
        self.dataset["T5"] = ['A', 'B', 'C', 'D']
        
    def test0(self):
        frequent_1_itemsets = generate_frequent_itemsets(self.dataset, self.min_support, self.items)
        print (frequent_1_itemsets)
        frequent_1_itemsets_solution = dict()
        frequent_1_itemsets_solution['A'] = 3
        frequent_1_itemsets_solution['B'] = 5
        frequent_1_itemsets_solution['D'] = 4

        print ("Test 1: frequent 1 itemsets")
        assert frequent_1_itemsets == frequent_1_itemsets_solution

        frequent_2_itemsets = generate_frequent_itemsets(self.dataset, self.min_support, self.items, 2, frequent_1_itemsets)
        print (frequent_2_itemsets)
        frequent_2_itemsets_solution = dict()
        frequent_2_itemsets_solution[('A', 'B')] = 3
        frequent_2_itemsets_solution[('B', 'D')] = 4
        
        print ("Test 1: frequent 2 itemsets")
        assert frequent_2_itemsets == frequent_2_itemsets_solution

        frequent_3_itemsets = generate_frequent_itemsets(self.dataset, self.min_support, self.items, 3, frequent_2_itemsets)
        print (frequent_3_itemsets)
        frequent_3_itemsets_solution = dict()

        print ("Test 1: frequent 3 itemsets")
        assert frequent_3_itemsets == frequent_3_itemsets_solution         
   
tests = TestX()
tests_to_run = unittest.TestLoader().loadTestsFromModule(tests)
unittest.TextTestRunner().run(tests_to_run)

.

{'A': 3, 'B': 5, 'D': 4}
Test 1: frequent 1 itemsets
{('A', 'B'): 3, ('B', 'D'): 4}
Test 1: frequent 2 itemsets
{}
Test 1: frequent 3 itemsets



----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


<unittest.runner.TextTestResult run=1 errors=0 failures=0>

### [10 points] Problem 2 - FP-Growth Implementation
***

A sample dataset has been provided to you in the './data/dataset.pickle' path. Here are the attributes for the dataset. Use this dataset to test your functions.

- Dataset should load the transactions in the form of a python dictionary where each key holds the transaction id and the value is a python list of the items purchased in that transaction. 
- An example transaction will have the following structure. If items A, C, D, F are purchased in transaction T3, this would appear as follows in the dictionary.

```python
transactions = {
   "T3": ["A", "C", "D", "F"]
}
```

Note:
- A sample dataset to test your code has been provided in the location "./data/dataset.pickle". Please maintain this as it would be necessary while grading.
- Do not change the variable names of the returned values.
- After calculating each of those values, assign them to the corresponding value that is being returned.

In [16]:
def item_support(dataset, min_support):
    
#   A helper function that returns the support count of each item in the dataset.
#   Input:
#       1. dataset - A python dictionary containing the transactions. 
#       2. min_support - A floating point variable representing the min_support value for the set of transactions.
#   Output:
#       1. support_dict - A dictionary representing the support count of each item in the dataset.
    
    len_transactions = len(dataset)
    support_dict = dict()
    for key, value in dataset.items():
        
        # your code here
        for item in value:
            if item in support_dict:
                support_dict[item] += 1
            else:
                support_dict[item] = 1
    
    ### For reference only
    sorted_support = dict(sorted(support_dict.items(), key=lambda item: item[1], reverse=True))
    pruned_support = {key:val for key, val in sorted_support.items() if val/len_transactions >= min_support}
    ###
    
    return support_dict

In [17]:
# This cell has visible test cases that you can run to see if you are on the right track!
# Note: hidden tests will also be applied on other datasets for final grading.

dataset = dict()
with open('./data/dataset.pickle', 'rb') as handle:
    dataset = pickle.load(handle)

support_dict = item_support(dataset, 0.5)
support_dict_expected = {'C': 7, 'D': 9, 'E': 5, 'B': 6, 'A': 6}

print(f'The expected support_dict value for the given dataset is: {support_dict_expected}')
print(f'Your support_dict value is: {support_dict}')

try:
    assert support_dict == support_dict_expected
    print("Visible tests passed!")
except:
    print("Visible tests failed!")

The expected support_dict value for the given dataset is: {'C': 7, 'D': 9, 'E': 5, 'B': 6, 'A': 6}
Your support_dict value is: {'C': 7, 'D': 9, 'E': 5, 'B': 6, 'A': 6}
Visible tests passed!


In [18]:

# This cell has hidden test cases that will run after you submit your assignment. 


In [19]:
def reorder_transactions(dataset, min_support):
    
#   A helper function that reorders the transaction items based on maximum support count. It is important that you finish
#   the code in the previous cells since this function makes use of the support count dictionary calculated above.
#   Input:
#       1. dataset - A python dictionary containing the transactions. 
#       2. min_support - A floating point variable representing the min_support value for the set of transactions.
#   Output:
#       1. updated_dataset - A dictionary representing the transaction items in sorted order of their support counts.

    pruned_support = item_support(dataset, min_support) 
    updated_dataset = dict()
    
    # This loop sorts the transaction items based on the item support counts
    for key, value in dataset.items():
        updated_dataset[key] = sorted(value, key=pruned_support.get, reverse=True)
    
    # Update the following loop to remove items that do not belong to the pruned_support dictionary
    for key, value in updated_dataset.items():
        updated_values = list()
        for item in value:
            # your code here
            if item in pruned_support:
                updated_values.append(item)
        updated_dataset[key] = updated_values

            
        updated_dataset[key] = updated_values

    return updated_dataset

In [20]:
# This cell has visible test cases that you can run to see if you are on the right track!
# Note: hidden tests will also be applied on other datasets for final grading.

import pprint
import json
pp = pprint.PrettyPrinter(depth=4)

dataset = dict()
with open('./data/dataset.pickle', 'rb') as handle:
    dataset = pickle.load(handle)

updated_dataset = reorder_transactions(dataset, 0.5)
updated_dataset_expected = {'T1': ['D', 'C', 'E'], 'T2': ['D', 'C', 'B'], 'T3': ['D', 'C', 'A'],
                            'T4': ['D', 'C', 'A', 'E'], 'T5': ['D', 'C', 'A', 'B'], 'T6': ['B'],
                            'T7': ['D', 'E'], 'T8': ['D', 'C', 'A', 'B'], 'T9': ['D', 'A', 'B', 'E'], 'T10': ['D', 'C', 'A', 'B', 'E']}

print(f'The expected updated_dataset value for the given dataset is:')
pp.pprint(updated_dataset_expected)
print(f'Your updated_dataset value is:')
pp.pprint(updated_dataset)

try:
    assert updated_dataset == updated_dataset_expected
    print("Visible tests passed!")
except:
    print("Visible tests failed!")

The expected updated_dataset value for the given dataset is:
{'T1': ['D', 'C', 'E'],
 'T10': ['D', 'C', 'A', 'B', 'E'],
 'T2': ['D', 'C', 'B'],
 'T3': ['D', 'C', 'A'],
 'T4': ['D', 'C', 'A', 'E'],
 'T5': ['D', 'C', 'A', 'B'],
 'T6': ['B'],
 'T7': ['D', 'E'],
 'T8': ['D', 'C', 'A', 'B'],
 'T9': ['D', 'A', 'B', 'E']}
Your updated_dataset value is:
{'T1': ['D', 'C', 'E'],
 'T10': ['D', 'C', 'A', 'B', 'E'],
 'T2': ['D', 'C', 'B'],
 'T3': ['D', 'C', 'A'],
 'T4': ['D', 'C', 'A', 'E'],
 'T5': ['D', 'C', 'A', 'B'],
 'T6': ['B'],
 'T7': ['D', 'E'],
 'T8': ['D', 'C', 'A', 'B'],
 'T9': ['D', 'A', 'B', 'E']}
Visible tests passed!


In [21]:

# This cell has hidden test cases that will run after you submit your assignment. 


In [22]:
def build_fp_tree(updated_dataset):
    
#   Input: 
#       1. updated_dataset - A python dictionary containing the updated set of transactions based on the pruned support dictionary.
#   Output:
#       1. fp_tree - A dictionary representing the fp_tree. Each node should have a count and children attribute.
#        
#   HINT:
#       1. Loop over each transaction in the dataset and make an update to the fp_tree dictionary. 
#       2. For each loop iteration store a pointer to the previously visited node and update it's children in the next pass.
#       3. Update the root pointer when you start processing items in each transaction.
#       4. Reset the root pointer for each transaction.
#
#   Sample Tree Output:
#   {'Y': {'count': 3, 'children': {'V': {'count': 1, 'children': {}}}},
#    'X': {'count': 2, 'children': {'R': {'count': 1, 'children': {'F': {'count': 1, 'children': {}}}}}}}
    
    fp_tree = dict()
    for key, value in updated_dataset.items():
        
        # your code here
        current_node = fp_tree
        for item in value:
            if item not in current_node:
                # If the item is not a child of the current node, create it
                current_node[item] = {'count': 1, 'children': {}}
            else:
                # If the item already exists, just increment the count
                current_node[item]['count'] += 1
            # Move to the child node for the next item
            current_node = current_node[item]['children']
    return fp_tree

In [23]:
# This cell has visible test cases that you can run to see if you are on the right track!
# Note: hidden tests will also be applied on other datasets for final grading.

import pprint
pp = pprint.PrettyPrinter(depth=8)

dataset = dict()
with open('./data/dataset.pickle', 'rb') as handle:
    dataset = pickle.load(handle)

updated_dataset = reorder_transactions(dataset, 0.5)

fp_tree = build_fp_tree(updated_dataset)
fp_tree_expected = {'D': {'count': 9,
  'children': {'C': {'count': 7,
    'children': {'E': {'count': 1, 'children': {}},
     'B': {'count': 1, 'children': {}},
     'A': {'count': 5,
      'children': {'E': {'count': 1, 'children': {}},
       'B': {'count': 3, 'children': {'E': {'count': 1, 'children': {}}}}}}}},
   'E': {'count': 1, 'children': {}},
   'A': {'count': 1,
    'children': {'B': {'count': 1,
      'children': {'E': {'count': 1, 'children': {}}}}}}}},
 'B': {'count': 1, 'children': {}}}

print(f'The expected fp_tree value for the given dataset is:')
pp.pprint(fp_tree_expected)
print(f'\nYour fp_tree value is:')
pp.pprint(fp_tree)

try:
    assert fp_tree == fp_tree_expected
    print("Visible tests passed!")
except:
    print("Visible tests failed!")

The expected fp_tree value for the given dataset is:
{'B': {'children': {}, 'count': 1},
 'D': {'children': {'A': {'children': {'B': {'children': {'E': {'children': {},
                                                                'count': 1}},
                                             'count': 1}},
                          'count': 1},
                    'C': {'children': {'A': {'children': {'B': {'children': {'E': {'children': {},
                                                                                   'count': 1}},
                                                                'count': 3},
                                                          'E': {'children': {},
                                                                'count': 1}},
                                             'count': 5},
                                       'B': {'children': {}, 'count': 1},
                                       'E': {'children': {}, 'count': 1}},
                 

In [18]:

# This cell has hidden test cases that will run after you submit your assignment. 
